# 12 - Local and TAXII Modes (STIX 2.0 / 2.1)

This notebook shows how to:
- download ATT&CK STIX bundles in **STIX 2.0** and **STIX 2.1**
- load them locally (offline) with `AttackClient.from_local()`
- optionally connect to the MITRE ATT&CK **TAXII 2.1** server with `AttackClient.from_taxii()`

Notes:
- The MITRE TAXII server is **rate limited** (10 requests / 10 minutes / IP).
- Local bundles can be STIX 2.0 (cti repo) or STIX 2.1 (attack-stix-data).


## Imports
This uses the `attackcti` downloader and client.

In [ ]:
from __future__ import annotations

from pathlib import Path

from stix2 import Filter

from attackcti import MitreAttackClient
from attackcti.utils.downloader import STIXDownloader


## Download STIX 2.0 and STIX 2.1 bundles
These calls require network access. If you already have the bundles on disk, skip this section and set the paths below.

In [ ]:
download_root = Path('./downloads')
download_root.mkdir(parents=True, exist_ok=True)

# STIX 2.0 example (from mitre/cti tags)
stix20 = STIXDownloader(download_dir=str(download_root / 'stix-2.0'), stix_version='2.0')
stix20.download_attack_data(domain='enterprise', release='15.1', pretty_print=True)
stix20_enterprise = Path(stix20.downloaded_file_paths['enterprise'])

# STIX 2.1 example (from mitre-attack/attack-stix-data)
stix21 = STIXDownloader(download_dir=str(download_root / 'stix-2.1'), stix_version='2.1')
stix21.download_attack_data(domain='enterprise', release='18.1', pretty_print=True)
stix21_enterprise = Path(stix21.downloaded_file_paths['enterprise'])

stix20_enterprise, stix21_enterprise


## Load locally (offline)
`AttackClient.from_local()` loads STIX JSON bundles from disk and does not contact the network.

In [ ]:
# STIX 2.0 local mode (single file example)
client20 = MitreAttackClient.from_local(enterprise=str(stix20_enterprise))
print('mode:', client20.mode, 'spec_version:', client20.spec_version)

# Direct datastore query (works for both 2.0 and 2.1 local bundles)
techniques_20 = client20.TC_ENTERPRISE_SOURCE.query([Filter('type', '=', 'attack-pattern')])
print('enterprise attack-pattern count (2.0):', len(techniques_20))


In [ ]:
# STIX 2.1 local mode
# If you downloaded only enterprise above, you can still load just enterprise.
client21 = MitreAttackClient.from_local(enterprise=str(stix21_enterprise))
print('mode:', client21.mode, 'spec_version:', client21.spec_version)

techniques_21 = client21.TC_ENTERPRISE_SOURCE.query([Filter('type', '=', 'attack-pattern')])
print('enterprise attack-pattern count (2.1):', len(techniques_21))


## Connect to MITRE ATT&CK TAXII 2.1 (rate limited)
This is the network-backed mode. Keep queries small to avoid hitting the rate limit.

In [ ]:
# TAXII mode (STIX 2.1 over TAXII 2.1)
taxii = MitreAttackClient.from_taxii()
print('mode:', taxii.mode, 'spec_version:', taxii.spec_version)

# Example: fetch one object by STIX id (one request)
stix_id = 'attack-pattern--ad255bfe-a9e6-4b52-a258-8d3462abe842'
obj = taxii.TC_ENTERPRISE_SOURCE.get(stix_id)
print(obj['type'], obj['id'], obj.get('spec_version'))


In [ ]:
obj